# <center> Домашнее задание №9. Решение
## <center> Анализ временных рядов

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from plotly import graph_objs as go

In [ ]:
def plotly_df(df, title = ''):
    data = []
    for column in df.columns:
        trace = go.Scatter(
            x = df.index,
            y = df[column],
            mode = 'lines',
            name = column
        )
        data.append(trace)
    layout = dict(title = title)
    fig = go.Figure(data=data, layout=layout)
    fig.show()

## Подготовка данных

In [ ]:
df = pd.read_csv('../data/wiki_machine_learning.csv', sep = ' ')
df = df[df['count'] != 0]
df.head()

In [ ]:
df.shape

## Предсказание с помощью FB Prophet
Обучим модель на первых 5 месяцах и предскажем количество посещений за июнь.

In [ ]:
df.date = pd.to_datetime(df.date)

In [ ]:
plotly_df(df.set_index('date')[['count']])

In [ ]:
from prophet import Prophet

In [ ]:
predictions = 30

df = df[['date', 'count']]
df.columns = ['ds', 'y']
df.tail()

In [ ]:
train_df = df[:-predictions].copy()

In [ ]:
m = Prophet()
m.fit(train_df);

In [ ]:
future = m.make_future_dataframe(periods=predictions)
future.tail()

In [ ]:
forecast = m.predict(future)
forecast.tail()

**<font color='red'>Вопрос 1:</font>** Каково предсказание количества просмотров wiki-страницы на 20 января? Округлите до ближайшего целого.

- 4947
- 3426 **[+]**
- 5229
- 2744

In [ ]:
m.plot(forecast)

In [ ]:
m.plot_components(forecast)

In [ ]:
cmp_df = forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(df.set_index('ds'))

In [ ]:
import numpy as np
cmp_df['e'] = cmp_df['y'] - cmp_df['yhat']
cmp_df['p'] = 100 * cmp_df['e'] / cmp_df['y']
print('MAPE = ', round(np.mean(abs(cmp_df[-predictions:]['p'])), 2))
print('MAE = ', round(np.mean(abs(cmp_df[-predictions:]['e'])), 2))

Оценим качество предсказания на последних 30 точках.

**<font color='red'>Вопрос 2:</font> Чему равен MAPE?**

- 34.5 **[+]**
- 42.42
- 5.39
- 65.91

**<font color='red'>Вопрос 3:</font> Чему равен MAE?**

- 355
- 4007
- 600 **[+]**
- 903

## Предсказание с помощью ARIMA

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
plt.rcParams['figure.figsize'] = (15, 10)

**<font color='red'>Вопрос 4:</font> Проверим стационарность ряда с помощью теста Дики-Фуллера. Является ли ряд стационарным? Каково p-value?**

- Ряд стационарен, p_value = 0.107
- Ряд не стационарен, p_value = 0.107 **[+]**
- Ряд стационарен, p_value = 0.001
- Ряд не стационарен, p_value = 0.001

In [ ]:
sm.tsa.seasonal_decompose(train_df['y'].values, period=7).plot();
print("Тест Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(train_df['y'])[1])

Но сезонно дифференцированный ряд уже будет стационарным.

In [ ]:
train_df.set_index('ds', inplace=True)

In [ ]:
train_df['y_diff'] = train_df.y - train_df.y.shift(7)
sm.tsa.seasonal_decompose(train_df.y_diff[7:].values, period=7).plot();
print("Тест Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(train_df.y_diff[8:])[1])

In [ ]:
ax = plt.subplot(211)
sm.graphics.tsa.plot_acf(train_df.y_diff[13:].values.squeeze(), lags=48, ax=ax)

ax = plt.subplot(212)
sm.graphics.tsa.plot_pacf(train_df.y_diff[13:].values.squeeze(), lags=48, ax=ax)

Начальные значения:
* Q = 1
* q = 3
* P = 3
* p = 1

In [ ]:
ps = range(0, 2)
ds = range(0, 2)
qs = range(0, 4)
Ps = range(0, 4)
Ds = range(0, 3)
Qs = range(0, 2)

In [ ]:
from itertools import product

parameters = product(ps, ds, qs, Ps, Ds, Qs)
parameters_list = list(parameters)
len(parameters_list)

In [ ]:
%%time
import warnings
from tqdm import tqdm
results1 = []
best_aic = float("inf")
warnings.filterwarnings('ignore')

for param in tqdm(parameters_list):
    # try/except необходим, т.к. на некоторых наборах параметров модель не обучается
    try:
        model=sm.tsa.statespace.SARIMAX(train_df['y'], order=(param[0], param[1], param[2]), 
                                        seasonal_order=(param[3], param[4], param[5], 7)).fit(disp=-1)
    # выводим параметры, на которых модель не обучилась, и переходим к следующему набору
    except (ValueError, np.linalg.LinAlgError):
        continue
    aic = model.aic
    # сохраняем лучшую модель, aic, параметры
    if aic < best_aic:
        best_model = model
        best_aic = aic
        best_param = param
    results1.append([param, model.aic])

In [ ]:
result_table1 = pd.DataFrame(results1)
result_table1.columns = ['parameters', 'aic']
print(result_table1.sort_values(by = 'aic', ascending=True).head())

Если рассмотреть варианты, предложенные в задании:

In [ ]:
result_table1[result_table1['parameters'].isin([(1, 0, 2, 3, 1, 0),
                                                (1, 1, 2, 3, 2, 1),
                                                (1, 1, 2, 3, 1, 1),
                                                (1, 0, 2, 3, 0, 0)])]

Теперь сделаем то же самое, но для ряда с преобразованием Бокса-Кокса.

In [ ]:
import scipy.stats
train_df['y_box'], lmbda = scipy.stats.boxcox(train_df['y']) 
print("Оптимальный параметр преобразования Бокса-Кокса: %f" % lmbda)

In [ ]:
results2 = []
best_aic = float("inf")

for param in tqdm(parameters_list):
    # try/except необходим, т.к. на некоторых наборах параметров модель не обучается
    try:
        model=sm.tsa.statespace.SARIMAX(train_df['y_box'], order=(param[0], param[1], param[2]), 
                                        seasonal_order=(param[3], param[4], param[5], 7)).fit(disp=-1)
    # выводим параметры, на которых модель не обучилась, и переходим к следующему набору
    except (ValueError, np.linalg.LinAlgError):
        continue
    aic = model.aic
    # сохраняем лучшую модель, aic, параметры
    if aic < best_aic:
        best_model = model
        best_aic = aic
        best_param = param
    results2.append([param, model.aic])
    
warnings.filterwarnings('default')

In [ ]:
result_table2 = pd.DataFrame(results2)
result_table2.columns = ['parameters', 'aic']
print(result_table2.sort_values(by = 'aic', ascending=True).head())

Если рассмотреть варианты, предложенные в задании:

In [ ]:
result_table2[result_table2['parameters'].isin([(1, 0, 2, 3, 1, 0),
                                                (1, 1, 2, 3, 2, 1),
                                                (1, 1, 2, 3, 1, 1),
                                                (1, 0, 2, 3, 0, 0)])].sort_values(by='aic')

**Далее перейдём к построению модели SARIMAX (`sm.tsa.statespace.SARIMAX`).<br> <font color='red'>Вопрос 5:</font> Какие параметры оптимальны для модели по критерию `AIC`?**

- D = 1, d = 0, Q = 0, q = 2, P = 3, p = 1
- D = 2, d = 1, Q = 1, q = 2, P = 3, p = 1 **[+]**
- D = 1, d = 1, Q = 1, q = 2, P = 3, p = 1
- D = 0, d = 0, Q = 0, q = 2, P = 3, p = 1

Посмотрим на прогноз лучшей модели по AIC.

In [ ]:
print(best_model.summary())

In [ ]:
plt.subplot(211)
best_model.resid[13:].plot()
plt.ylabel('Остатки')

ax = plt.subplot(212)
sm.graphics.tsa.plot_acf(best_model.resid[13:].values.squeeze(), lags=48, ax=ax)

print("t-тест Стьюдента: p=%f" % stats.ttest_1samp(best_model.resid[13:], 0)[1])
print("Тест Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(best_model.resid[13:])[1])

In [ ]:
def invboxcox(y, lmbda):
    # обратное преобразование Бокса-Кокса
    if lmbda == 0:
        return(np.exp(y))
    else:
        return(np.exp(np.log(lmbda * y + 1) / lmbda))

In [ ]:
train_df['arima_model'] = invboxcox(best_model.fittedvalues, lmbda)

train_df.y.tail(200).plot()
train_df.arima_model[13:].tail(200).plot(color='r')
plt.ylabel('Просмотры wiki-страницы');